In [2]:
%pip install gensim
#%pip install pyhealth


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
#import pyhealth
import os
import warnings
import sagemaker
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Input
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
#from sagemaker.tensorflow import TensorFlow


2023-05-07 04:30:29.023915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 04:30:29.150391: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-05-07 04:30:29.150481: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-05-07 04:30:29.187495: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-07 04:30:29.195740: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the Sage

We will first import the data needed for this experiment

In [4]:
#1. The PubMed word2vec vectors
w2vec_url = 'http://evexdb.org/pmresources/vec-space-models/'
w2vec_filename = 'PubMed-and-PMC-w2v.bin'
if not os.path.isfile('./data/' + w2vec_filename):
    !cd data
    !wget {w2vec_url}{w2vec_filename}

In [5]:
#2. The admissions records from MIMIC III

admission_df = pd.read_csv(os.getcwd() + '/data/ADMISSIONS.csv.gz')
admission_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [6]:
#3. The ICD Diagnoses records from MIMIC III
diagnoses_df = pd.read_csv(os.getcwd() + '/data/DIAGNOSES_ICD.csv.gz')
diagnoses_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [7]:
%%time
#4. The notes records from MIMIC III

notes_df = pd.read_csv(os.getcwd() + '/data/NOTEEVENTS.csv.gz')
notes_df.head()

<timed exec>:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 53.8 s, sys: 2.48 s, total: 56.3 s
Wall time: 56.3 s


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [8]:
# ICD-9 codes for heart failure. The paper had the origional ICD numbers with decimel in place. The MIMIC dataset removed all decimels.
# The codes below match the MIMIC III ICD codes (i.e. without decimals)
hf_icd9_codes = [
    '39891', '40201', '40211', '40291', '40401', '40403', '40411',
    '40413', '40491', '40493', '4280', '4281', '42820', '42821',
    '42822', '42823', '42830', '42831', '42832', '42833', '42840',
    '42841', '42842', '42843', '4289'
]

In [9]:
#Filter out all diagnoses that does not pertain to heart failure diagnoses
hf_diagnoses_df = diagnoses_df[diagnoses_df['ICD9_CODE'].isin(hf_icd9_codes)]
hf_diagnoses_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
51,1511,115,114585,10.0,4280
67,1527,117,140784,5.0,4280
150,1610,124,138376,12.0,42833
153,1613,124,138376,15.0,4280
211,1671,130,198214,2.0,4280


In [10]:
#filter out all the notes that are note of the category "Discharge summary". Also cleaned converted type of HADM_ID in notes_df becuase they
#were imported as floats while other other MIMIC III data imported this column as ints

discharge_notes_df = notes_df[notes_df["CATEGORY"] == "Discharge summary"]
discharge_notes_df = discharge_notes_df.round().astype({'HADM_ID':'int64'})
discharge_notes_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [11]:
#Create one dataframe that has all the data we want
intermediate_df = admission_df.merge(discharge_notes_df, how="left", on="HADM_ID")
admission_diagnoses_notes_df = intermediate_df.merge(hf_diagnoses_df, how="left", on="HADM_ID")
admission_diagnoses_notes_df.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2196-4-9**] Discharg...,NaN,NaN,NaN,NaN
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2153-9-3**] D...,NaN,NaN,NaN,NaN
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2157-10-18**] ...,NaN,NaN,NaN,NaN
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2139-6-6**] D...,NaN,NaN,NaN,NaN
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2160-11-2**] ...,NaN,NaN,NaN,NaN


In [12]:
#Drop all rows where the patient was NOT diagnosed with a heart failure related ICD9 code
admission_diagnoses_notes_df = admission_diagnoses_notes_df.dropna(subset=["ICD9_CODE"]).reset_index()
admission_diagnoses_notes_df.head()

,index,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,...,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2126-5-6**] Discharge ...,186.0,26.0,3.0,4280
1,8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2172-10-14**] Discha...,211.0,30.0,4.0,4281
2,12,33,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,...,NaN,NaN,NaN,NaN,NaN,NaN,237.0,34.0,2.0,4280
3,13,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-2-23**] ...,245.0,34.0,2.0,42822
4,14,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-2-23**] ...,249.0,34.0,6.0,4280


In [13]:
#Drop all rows where a discharge summary does not exist
admission_diagnoses_notes_df= admission_diagnoses_notes_df.dropna(subset=["CATEGORY"]).reset_index()
admission_diagnoses_notes_df.head(2)

,level_0,index,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,...,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,0,5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2126-5-6**] Discharge ...,186.0,26.0,3.0,4280
1,1,8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2172-10-14**] Discha...,211.0,30.0,4.0,4281


In [14]:
admission_diagnoses_notes_df = admission_diagnoses_notes_df.drop(["level_0", "index"], axis=1)
admission_diagnoses_notes_df.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2126-5-6**] Discharge ...,186.0,26.0,3.0,4280
1,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2172-10-14**] Discha...,211.0,30.0,4.0,4281
2,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-2-23**] ...,245.0,34.0,2.0,42822
3,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-2-23**] ...,249.0,34.0,6.0,4280
4,39,37,188670,2183-08-21 16:48:00,2183-08-26 18:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2183-8-21**] Discharge...,297.0,37.0,2.0,4280


In [15]:
admission_diagnoses_notes_df["ADMITTIME"] = pd.to_datetime(admission_diagnoses_notes_df["ADMITTIME"])
admission_diagnoses_notes_df["DISCHTIME"] = pd.to_datetime(admission_diagnoses_notes_df["DISCHTIME"])
admission_diagnoses_notes_df.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2126-5-6**] Discharge ...,186.0,26.0,3.0,4280
1,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2172-10-14**] Discha...,211.0,30.0,4.0,4281
2,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-2-23**] ...,245.0,34.0,2.0,42822
3,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-2-23**] ...,249.0,34.0,6.0,4280
4,39,37,188670,2183-08-21 16:48:00,2183-08-26 18:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2183-8-21**] Discharge...,297.0,37.0,2.0,4280


In [16]:
admission_diagnoses_notes_df["SAME_SUBJECT_ID"] = admission_diagnoses_notes_df["SUBJECT_ID_x"] == admission_diagnoses_notes_df["SUBJECT_ID_x"].shift(1)
admission_diagnoses_notes_df["SAME_HADM_ID"] = admission_diagnoses_notes_df["HADM_ID"] == admission_diagnoses_notes_df["HADM_ID"].shift(1)
admission_diagnoses_notes_df.head(2)

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,DESCRIPTION,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID
0,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,...,Report,NaN,NaN,Admission Date: [**2126-5-6**] Discharge ...,186.0,26.0,3.0,4280,False,False
1,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,Report,NaN,NaN,Admission Date: [**2172-10-14**] Discha...,211.0,30.0,4.0,4281,False,False


In [17]:
readmit_times = []

subject_id = 0
hadm_id = 0
dischtime = pd.Timedelta(0)
last_time = pd.Timedelta(0)
for i in range(len(admission_diagnoses_notes_df)):
    
    if admission_diagnoses_notes_df["SUBJECT_ID_x"][i] != subject_id:
        subject_id = admission_diagnoses_notes_df["SUBJECT_ID_x"][i]
        hadm_id = admission_diagnoses_notes_df["HADM_ID"][i]
        dischtime = admission_diagnoses_notes_df["DISCHTIME"][i]
        
        readmit_times.append(pd.Timedelta(0))
        
    elif (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] == subject_id) and (admission_diagnoses_notes_df["HADM_ID"][i] == hadm_id) and (readmit_times[-1] == pd.Timedelta(0)):
        readmit_times.append(pd.Timedelta(0))
        
    elif (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] == subject_id) and (admission_diagnoses_notes_df["HADM_ID"][i] != hadm_id):
        readmit_times.append(admission_diagnoses_notes_df["ADMITTIME"][i] - dischtime)
        
        last_time = admission_diagnoses_notes_df["ADMITTIME"][i] - dischtime
        hadm_id = admission_diagnoses_notes_df["HADM_ID"][i]
        dischtime = admission_diagnoses_notes_df["DISCHTIME"][i]
        
    elif (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] == subject_id) and (admission_diagnoses_notes_df["HADM_ID"][i] == hadm_id):
        readmit_times.append(last_time)
        
    else:
        readmit_times.append(pd.Timedelta(0))

In [18]:
admission_diagnoses_notes_df["READMISSION_TIMES"] = readmit_times

In [19]:
admission_diagnoses_notes_df[admission_diagnoses_notes_df["SUBJECT_ID_x"] == 357]

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,CGID,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES
6,456,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,NaN,NaN,Admission Date: [**2197-12-6**] ...,4334.0,357.0,8.0,4280,False,False,0 days 00:00:00
7,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,NaN,NaN,Admission Date: [**2198-11-1**] ...,4297.0,357.0,14.0,4280,True,False,302 days 08:36:00
8,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,NaN,NaN,Admission Date: [**2198-11-1**] ...,4298.0,357.0,15.0,42823,True,True,302 days 08:36:00
9,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,NaN,Admission Date: [**2199-10-20**] ...,4246.0,357.0,2.0,42822,True,False,339 days 21:45:00
10,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,NaN,Admission Date: [**2199-10-20**] ...,4253.0,357.0,9.0,4280,True,True,339 days 21:45:00
11,460,357,117876,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,NaN,Admission Date: [**2199-12-21**] ...,4271.0,357.0,10.0,42822,True,False,59 days 05:07:00
12,460,357,117876,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,NaN,Admission Date: [**2199-12-21**] ...,4273.0,357.0,12.0,4280,True,True,59 days 05:07:00


In [20]:
general_readmission = []

subject_id = 0
hadm_id = 0
for i in range(len(admission_diagnoses_notes_df) - 1, -1, -1):
        
    if (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] != subject_id):
        subject_id = admission_diagnoses_notes_df["SUBJECT_ID_x"][i]
        hadm_id = admission_diagnoses_notes_df["HADM_ID"][i]
        
        general_readmission.append(0)
        
    elif (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] == subject_id) and (admission_diagnoses_notes_df["HADM_ID"][i] != hadm_id):
        general_readmission.append(1)
        
        
    else:
        general_readmission.append(0)

In [21]:
admission_diagnoses_notes_df["GENERAL_READMISSION"] = list(reversed(general_readmission))

In [22]:
admission_diagnoses_notes_df[admission_diagnoses_notes_df["SUBJECT_ID_x"] == 357]

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ISERROR,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION
6,456,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,NaN,Admission Date: [**2197-12-6**] ...,4334.0,357.0,8.0,4280,False,False,0 days 00:00:00,1
7,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,NaN,Admission Date: [**2198-11-1**] ...,4297.0,357.0,14.0,4280,True,False,302 days 08:36:00,1
8,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,NaN,Admission Date: [**2198-11-1**] ...,4298.0,357.0,15.0,42823,True,True,302 days 08:36:00,1
9,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,Admission Date: [**2199-10-20**] ...,4246.0,357.0,2.0,42822,True,False,339 days 21:45:00,1
10,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,Admission Date: [**2199-10-20**] ...,4253.0,357.0,9.0,4280,True,True,339 days 21:45:00,1
11,460,357,117876,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,Admission Date: [**2199-12-21**] ...,4271.0,357.0,10.0,42822,True,False,59 days 05:07:00,0
12,460,357,117876,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,NaN,Admission Date: [**2199-12-21**] ...,4273.0,357.0,12.0,4280,True,True,59 days 05:07:00,0


In [23]:
thirtyday_readmission = []

subject_id = 0
hadm_id = 0
for i in range(len(admission_diagnoses_notes_df) - 1, -1, -1):
        
    if (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] != subject_id):
        subject_id = admission_diagnoses_notes_df["SUBJECT_ID_x"][i]
        hadm_id = admission_diagnoses_notes_df["HADM_ID"][i]
        
        thirtyday_readmission.append(0)
        
    elif (admission_diagnoses_notes_df["SUBJECT_ID_x"][i] == subject_id) and (admission_diagnoses_notes_df["HADM_ID"][i] != hadm_id) and (admission_diagnoses_notes_df["READMISSION_TIMES"][i] < pd.Timedelta(days=30)) and (admission_diagnoses_notes_df["READMISSION_TIMES"][i] > pd.Timedelta(days=0)):
        thirtyday_readmission.append(1)
        
    else:
        thirtyday_readmission.append(0)

In [24]:
admission_diagnoses_notes_df["THIRTYDAY_READMISSION"] = list(reversed(thirtyday_readmission))

In [25]:
admission_diagnoses_notes_df.shape[0]

23830

In [26]:
sum(admission_diagnoses_notes_df['GENERAL_READMISSION'])

5969

In [27]:
sum(admission_diagnoses_notes_df['THIRTYDAY_READMISSION'])

668

In [28]:
true_gen_readmit = admission_diagnoses_notes_df[admission_diagnoses_notes_df["GENERAL_READMISSION"] == True]

In [29]:
false_gen_readmit = admission_diagnoses_notes_df[admission_diagnoses_notes_df["GENERAL_READMISSION"] == False].sample(n=sum(admission_diagnoses_notes_df["GENERAL_READMISSION"]), random_state=1)

In [30]:
gen_readmission_dataset = pd.concat([true_gen_readmit, false_gen_readmit])
gen_readmission_dataset.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION,THIRTYDAY_READMISSION
6,456,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,Admission Date: [**2197-12-6**] ...,4334.0,357.0,8.0,4280,False,False,0 days 00:00:00,1,0
7,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,Admission Date: [**2198-11-1**] ...,4297.0,357.0,14.0,4280,True,False,302 days 08:36:00,1,0
8,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,Admission Date: [**2198-11-1**] ...,4298.0,357.0,15.0,42823,True,True,302 days 08:36:00,1,0
9,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,Admission Date: [**2199-10-20**] ...,4246.0,357.0,2.0,42822,True,False,339 days 21:45:00,1,0
10,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,Admission Date: [**2199-10-20**] ...,4253.0,357.0,9.0,4280,True,True,339 days 21:45:00,1,0


In [31]:
gen_readmission_dataset.shape[0]

11938

In [32]:
true_thirty_readmit = admission_diagnoses_notes_df[admission_diagnoses_notes_df["THIRTYDAY_READMISSION"] == True]

In [33]:
false_thirty_readmit = admission_diagnoses_notes_df[admission_diagnoses_notes_df["THIRTYDAY_READMISSION"] == False].sample(n=sum(admission_diagnoses_notes_df["THIRTYDAY_READMISSION"]), random_state=1)

In [34]:
thirty_readmission_dataset = pd.concat([true_thirty_readmit, false_thirty_readmit])
thirty_readmission_dataset.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION,THIRTYDAY_READMISSION
235,650,518,153168,2109-07-07 07:49:00,2109-07-09 12:16:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,Admission Date: [**2109-7-7**] D...,6159.0,518.0,4.0,4280,True,False,7 days 14:49:00,1,1
597,1908,1569,103678,2177-12-16 21:31:00,2178-01-08 13:52:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,Admission Date: [**2177-12-16**] ...,17589.0,1569.0,8.0,42830,True,False,4 days 01:01:00,1,1
598,1908,1569,103678,2177-12-16 21:31:00,2178-01-08 13:52:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,Admission Date: [**2177-12-16**] ...,17590.0,1569.0,9.0,4280,True,True,4 days 01:01:00,1,1
686,1650,1339,109444,2138-09-22 08:36:00,2138-09-30 15:25:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,Admission Date: [**2138-9-22**] ...,15234.0,1339.0,5.0,42833,True,False,1 days 20:06:00,1,1
687,1651,1339,158998,2138-10-13 11:24:00,2138-10-21 14:40:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,Admission Date: [**2138-10-11**] ...,15303.0,1339.0,2.0,42823,True,False,12 days 19:59:00,1,1


In [35]:
thirty_readmission_dataset.shape[0]

1336

In [36]:
gen_readmission_dataset = gen_readmission_dataset.sample(frac=1)
ratio = 0.9
 
total_rows = gen_readmission_dataset.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
train_gen = gen_readmission_dataset[0:train_size]
test_gen = gen_readmission_dataset[train_size:]

In [37]:
thirty_readmission_dataset = thirty_readmission_dataset.sample(frac=1)
ratio = 0.9
 
total_rows = thirty_readmission_dataset.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
train_thirty = thirty_readmission_dataset[0:train_size]
test_thirty = thirty_readmission_dataset[train_size:]

In [38]:
train_gen.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION,THIRTYDAY_READMISSION
8820,23993,19629,182981,2120-12-29 22:35:00,2121-01-14 17:25:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,...,Admission Date: [**2120-12-29**] ...,219321.0,19629.0,4.0,4280,False,False,0 days 00:00:00,0,0
22473,55436,89012,187553,2129-04-27 16:32:00,2129-05-01 15:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,...,Admission Date: [**2129-4-27**] ...,600774.0,89012.0,4.0,4280,True,True,0 days 00:00:00,0,0
13016,33824,27665,186634,2185-07-26 15:47:00,2185-09-14 12:15:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,...,"Name: [**Known lastname 9699**],[**Known firs...",312401.0,27665.0,2.0,42830,True,True,0 days 00:00:00,1,0
21820,54888,87251,125986,2154-08-09 16:53:00,2154-08-11 16:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Government,...,Admission Date: [**2154-8-9**] D...,593494.0,87251.0,2.0,42821,False,False,0 days 00:00:00,0,0
17199,44923,55992,161385,2153-08-19 13:04:00,2153-08-24 20:15:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,"Name: [**Known lastname 2104**],[**Known firs...",452903.0,55992.0,4.0,4280,True,True,17 days 17:19:00,1,1


In [39]:
test_gen.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION,THIRTYDAY_READMISSION
19930,46994,62413,135946,2133-02-09 07:15:00,2133-02-18 15:45:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,...,"Name: [**Known lastname 14043**],[**Known fir...",482043.0,62413.0,2.0,42822,True,True,0 days 00:00:00,0,0
16990,42723,49311,191046,2121-05-15 22:33:00,2121-05-19 14:18:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Medicare,...,Admission Date: [**2121-5-15**] ...,420907.0,49311.0,7.0,4280,True,True,1 days 08:54:00,1,1
3973,8893,7275,125765,2135-11-19 17:22:00,2135-11-20 21:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LEFT AGAINST MEDICAL ADVI,Medicare,...,Admission Date: [**2135-11-19**] ...,81266.0,7275.0,10.0,42830,True,True,381 days 03:22:00,1,0
8293,21287,17412,178656,2101-01-12 00:00:00,2101-01-21 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,...,Admission Date: [**2101-1-12**] ...,194131.0,17412.0,4.0,4280,False,False,0 days 00:00:00,1,0
17280,41408,45213,197986,2106-01-31 22:50:00,2106-02-07 12:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,...,Admission Date: [**2106-1-31**] ...,401985.0,45213.0,5.0,4280,True,True,0 days 00:00:00,1,0


In [40]:
train_thirty.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION,THIRTYDAY_READMISSION
6721,19223,15749,171145,2180-09-29 17:33:00,2180-10-14 13:40:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,Admission Date: [**2180-9-29**] ...,175221.0,15749.0,11.0,4280,True,True,1844 days 04:08:00,0,0
17289,41961,46904,103536,2136-06-07 18:42:00,2136-06-13 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,...,Admission Date: [**2136-6-7**] D...,409785.0,46904.0,16.0,4280,True,True,1 days 01:02:00,1,1
3106,9111,7449,135044,2138-11-03 18:06:00,2138-11-13 17:49:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,...,Admission Date: [**2138-11-3**] ...,83185.0,7449.0,5.0,4280,True,True,0 days 00:00:00,0,0
19721,48954,68457,118480,2129-08-23 04:43:00,2129-09-01 16:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Medicare,...,Admission Date: [**2129-8-23**] ...,510030.0,68457.0,9.0,4280,True,True,10 days 13:03:00,1,1
7906,23073,18921,150037,2193-06-27 18:37:00,2193-07-09 16:56:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,Admission Date: [**2193-6-27**] ...,210445.0,18921.0,1.0,4280,True,False,64 days 02:06:00,0,0


In [41]:
test_thirty.head()

,ROW_ID_x,SUBJECT_ID_x,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE,SAME_SUBJECT_ID,SAME_HADM_ID,READMISSION_TIMES,GENERAL_READMISSION,THIRTYDAY_READMISSION
756,1923,1578,110184,2140-12-22 15:54:00,2141-01-05 15:27:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,Admission Date: [**2140-12-22**] ...,17734.0,1578.0,7.0,4280,True,True,26 days 00:30:00,1,1
23284,57258,94698,102396,2106-01-03 13:02:00,2106-01-22 14:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,Admission Date: [**2106-1-3**] D...,626409.0,94698.0,9.0,42833,True,False,7 days 21:32:00,1,1
12931,36051,29558,169227,2109-08-09 20:33:00,2109-08-24 17:53:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,...,Admission Date: [**2109-8-9**] D...,338225.0,29558.0,23.0,4280,True,True,80 days 06:13:00,0,0
5063,13227,10814,141996,2163-07-12 18:13:00,2163-07-20 14:30:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Medicare,...,Admission Date: [**2163-7-12**] ...,121358.0,10814.0,2.0,42823,True,False,5 days 06:28:00,1,1
9519,23585,19296,181850,2155-05-27 17:44:00,2155-06-17 19:19:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,...,Admission Date: [**2155-5-27**] ...,215491.0,19296.0,3.0,4280,True,False,1048 days 06:09:00,0,0


Now to start building out the CNN. Defining a few functions that can be reused for ML models

In [42]:
def values_labels(training_set, test_set):
    train_texts = train_gen['TEXT'].values
    test_texts = test_gen['TEXT'].values
    train_labels = train_gen['GENERAL_READMISSION'].values
    test_labels = test_gen['GENERAL_READMISSION'].values
    return train_texts, test_texts, train_labels, test_labels


In [43]:
word2vec_path = os.getcwd() + '/data/PubMed-and-PMC-w2v.bin'
word2vec_model = KeyedVectors.load_word2vec_format(datapath(word2vec_path), binary=True)
embedding_dim = word2vec_model.vector_size

In [44]:
def build_embedding_info(training_texts, testing_texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(training_texts)
    train_sequences = tokenizer.texts_to_sequences(training_texts)
    test_sequences = tokenizer.texts_to_sequences(testing_texts)
    
    max_sequence_length = max([len(seq) for seq in train_sequences])
    train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
    test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')
    
    vocab_size = len(tokenizer.word_index) + 1

    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in tokenizer.word_index.items():
        if word in word2vec_model:
            embedding_matrix[i] = word2vec_model[word]
    return max_sequence_length, train_padded, test_padded, vocab_size, embedding_matrix

In [45]:
def build_CNN(max_sequence_length,embedding_dim, vocab_size, embedding_matrix):
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=True)(input_layer)
    conv1 = Conv1D(filters=128, kernel_size=1, activation='relu')(embedding_layer)
    conv2 = Conv1D(filters=128, kernel_size=2, activation='relu')(embedding_layer)
    conv3 = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding_layer)

    pool1 = GlobalMaxPooling1D()(conv1)
    pool2 = GlobalMaxPooling1D()(conv2)
    pool3 = GlobalMaxPooling1D()(conv3)

    concat = tf.concat([pool1, pool2, pool3], axis=-1)
    output_layer = Dense(1, activation='sigmoid')(concat)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [47]:
gen_train_texts, gen_test_texts, gen_train_labels, gen_test_labels = values_labels(train_gen, test_gen)
print(f'training text size={gen_train_texts.shape[0]}')
print(f'test text size={gen_test_texts.shape[0]}')
print(f'training labels size={gen_train_labels.shape[0]}')
print(f'test labels size={gen_test_labels.shape[0]}')

training text size=10744
test text size=1194
training labels size=10744
test labels size=1194


In [49]:
max_sequence_length, gen_train_padded, gen_test_padded, gen_vocab_size, gen_embedding_matrix = build_embedding_info(gen_train_texts, gen_test_texts)

In [50]:
gen_model = build_CNN(max_sequence_length, embedding_dim, gen_vocab_size, gen_embedding_matrix)

2023-05-07 04:35:04.585707: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/site-packages/smdistributed/dataparallel/lib:/usr/local/lib:/opt/amazon/openmpi/lib/:/opt/amazon/efa/lib/:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/lib
2023-05-07 04:35:04.585739: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-07 04:35:04.585761: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-10-1-gp-ml-m5-2xlarge-e1e5905768965f7ca9d3d68b6ae2): /proc/driver/nvidia/version does not exist
2023-05-07 04:35:04.586010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU 

Extension horovod.torch has not been built: /usr/local/lib/python3.9/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-39-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.
[2023-05-07 04:35:04.916 tensorflow-2-10-1-gp-ml-m5-2xlarge-e1e5905768965f7ca9d3d68b6ae2:126 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-05-07 04:35:05.133 tensorflow-2-10-1-gp-ml-m5-2xlarge-e1e5905768965f7ca9d3d68b6ae2:126 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


/usr/local/lib/python3.9/site-packages/smdebug-1.0.26b20230210-py3.9.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/usr/local/lib/python3.9/site-packages/smdebug-1.0.26b20230210-py3.9.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [51]:
%%time
gen_model.fit(gen_train_padded, gen_train_labels, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


2023-05-07 04:35:34.846512: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 329248788 exceeds 10% of free system memory.


303/303 [==============================] - 429s 1s/step - loss: 0.6137 - accuracy: 0.6459 - val_loss: 0.5463 - val_accuracy: 0.7088
Epoch 2/10
303/303 [==============================] - 426s 1s/step - loss: 0.4707 - accuracy: 0.7873 - val_loss: 0.5364 - val_accuracy: 0.7219
Epoch 3/10
303/303 [==============================] - 425s 1s/step - loss: 0.3250 - accuracy: 0.8891 - val_loss: 0.4982 - val_accuracy: 0.7312
Epoch 4/10
303/303 [==============================] - 425s 1s/step - loss: 0.1801 - accuracy: 0.9623 - val_loss: 0.4176 - val_accuracy: 0.8009
Epoch 5/10
303/303 [==============================] - 425s 1s/step - loss: 0.0746 - accuracy: 0.9970 - val_loss: 0.4103 - val_accuracy: 0.7851
Epoch 6/10
303/303 [==============================] - 424s 1s/step - loss: 0.0290 - accuracy: 1.0000 - val_loss: 0.4230 - val_accuracy: 0.7898
Epoch 7/10
303/303 [==============================] - 424s 1s/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.4361 - val_accuracy: 0.7916
Epoch 8/10

In [53]:
loss, accuracy = gen_model.evaluate(gen_test_padded, gen_test_labels)
print(f'Test set accuracy: {accuracy}')

38/38 [==============================] - 11s 295ms/step - loss: 0.5118 - accuracy: 0.8074
Test set accuracy: 0.8073701858520508


In [55]:
#from tensorflow.keras.models import load_model

# Make predictions on the test set
test_predictions = gen_model.predict(gen_test_padded)

# Set a threshold to classify predictions as positive or negative
threshold = 0.5
predicted_labels = [1 if pred >= threshold else 0 for pred in test_predictions]

accuracy = accuracy_score(gen_test_labels, predicted_labels)
precision = precision_score(gen_test_labels, predicted_labels)
recall = recall_score(gen_test_labels, predicted_labels)
f1 = f1_score(gen_test_labels, predicted_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 score: {f1:.2f}")

38/38 [==============================] - 11s 294ms/step
Accuracy: 0.81
Precision: 0.79
Recall: 0.84
F1 score: 0.81


In [56]:
thirty_day_train_texts, thirty_day_test_texts, thirty_day_train_labels, thirty_day_test_labels = values_labels(train_thirty, test_thirty)
print(f'training text size={thirty_day_train_texts.shape[0]}')
print(f'test text size={thirty_day_test_texts.shape[0]}')
print(f'training labels size={thirty_day_train_labels.shape[0]}')
print(f'test labels size={thirty_day_test_labels.shape[0]}')

training text size=10744
test text size=1194
training labels size=10744
test labels size=1194


In [57]:
max_sequence_length, thirty_day_train_padded, thirty_day_test_padded, thirty_day_vocab_size, thirty_day_embedding_matrix = build_embedding_info(thirty_day_train_texts, thirty_day_test_texts)

In [58]:
thirty_day_model = build_CNN(max_sequence_length, embedding_dim, thirty_day_vocab_size, thirty_day_embedding_matrix)

In [59]:
%%time
thirty_day_model.fit(thirty_day_train_padded, thirty_day_train_labels, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


2023-05-07 05:59:53.232359: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 329248788 exceeds 10% of free system memory.


303/303 [==============================] - 424s 1s/step - loss: 0.6087 - accuracy: 0.6573 - val_loss: 0.5583 - val_accuracy: 0.6977
Epoch 2/10
303/303 [==============================] - 423s 1s/step - loss: 0.4632 - accuracy: 0.7876 - val_loss: 0.4983 - val_accuracy: 0.7544
Epoch 3/10
303/303 [==============================] - 423s 1s/step - loss: 0.3197 - accuracy: 0.8919 - val_loss: 0.4542 - val_accuracy: 0.7879
Epoch 4/10
303/303 [==============================] - 423s 1s/step - loss: 0.1752 - accuracy: 0.9662 - val_loss: 0.4307 - val_accuracy: 0.7823
Epoch 5/10
303/303 [==============================] - 423s 1s/step - loss: 0.0779 - accuracy: 0.9960 - val_loss: 0.4184 - val_accuracy: 0.7991
Epoch 6/10
303/303 [==============================] - 424s 1s/step - loss: 0.0315 - accuracy: 1.0000 - val_loss: 0.4265 - val_accuracy: 0.8065
Epoch 7/10
303/303 [==============================] - 424s 1s/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 0.4603 - val_accuracy: 0.7926
Epoch 8/10

KeyboardInterrupt: 

In [ ]:
loss, accuracy = thirty_day_model.evaluate(thirty_day_test_padded, thirty_day_test_labels)
print(f'Test set accuracy: {accuracy}')

In [ ]:

# Make predictions on the test set
test_predictions = thirty_day_model.predict(thirty_day_test_padded)

# Set a threshold to classify predictions as positive or negative
threshold = 0.5
predicted_labels = [1 if pred >= threshold else 0 for pred in test_predictions]

# Evaluate the model's performance using various metrics
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(thirty_day_test_labels, predicted_labels)
precision = precision_score(thirty_day_test_labels, predicted_labels)
recall = recall_score(thirty_day_test_labels, predicted_labels)
f1 = f1_score(thirty_day_test_labels, predicted_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 score: {f1:.2f}")